In [26]:
# Autoreload 
%load_ext autoreload
%autoreload 2

from pathlib import Path 
import pandas as pd 
import numpy as np
import os 
import re 
import gzip 
import shutil
import Bio.PDB.MMCIF2Dict
from typing import Union, List, Tuple, Dict, Optional
from pathlib import Path

pd.options.mode.chained_assignment = None  # default='warn'

from phosphosite.utils import aa1to3, aa3to1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
"""Initialise phosphosite dataset."""
from phosphosite.dataset import phosphorylation # Filtered out isoforms
df = phosphorylation[phosphorylation["ORGANISM"] == "human"]

# Sort by ACC_ID
df = df.sort_values("ACC_ID")

# Filter by residue type, first character in MOD_RSD
allowed_residues = "STY"
df = df[df["MOD_RSD"].str[0].isin(list(allowed_residues))]
phosphosite_df = df

In [5]:

from phosphosite import DATA_DIR
annotation_dir = DATA_DIR / "structure_annotations"

if False:
    N = 10000
    filepath = annotation_dir / f"structure_df_{N}.csv"
    # load df 
    structure_df = pd.read_csv(filepath, sep="\t")
    structure_df.head()
    # save df as HDF
    structure_df.to_hdf(annotation_dir / f"structure_df.h5", key="structure_df")

In [8]:
# Load from h5 
structure_df = pd.read_hdf(annotation_dir / f"structure_df.h5", key="structure_df")

In [156]:
# Filter to first 100 unique protein ids
all_protein_ids = list(structure_df["protein_id"].unique())
protein_ids = all_protein_ids[0:100]
subset_df = structure_df[structure_df["protein_id"].isin(protein_ids)]

In [157]:
len(all_protein_ids)

17336

In [25]:
def get_euc_dist(
    arr1: np.ndarray, arr2: np.ndarray
): 
    """Get euclidean distance between two arrays."""
    return np.sqrt(np.sum((arr1 - arr2) ** 2))
    

def get_node_id(
    site: str, 
    chain_id: str = "A",
) -> str: 
    mod_rsd, modification = site.split("-")
    aa = aa1to3[mod_rsd[0]]
    position = mod_rsd[1:]
    node_id = f"{chain_id}:{aa}:{position}"
    return node_id

def generate_node_id(
    node_dict: Dict[str, Union[str, int]],
    delimiter: str = ":",
) -> str: 
    return delimiter.join([str(node_dict[s]) for s in ["chain_id", "residue_name", "residue_number"]])

In [185]:
from tqdm import tqdm

"""Process motifs for a given set of protein ids."""
def process_sites(
    phosphosite_df: pd.DataFrame, 
    structure_df: pd.DataFrame,
    protein_ids: List[str],
    ref_atom: str = "ca", # ["c", "ca", "cb", "n"]
    radius: float = 6.0,
    adjacent_range: int = 1,
    next_nearest: int = 2,
    to_process: str = "all", # "p" for just phosphorylated.
    residue_type: str = "S", # just serine for now.
    verbose: bool = False,
    filepath: Optional[Union[str, Path]] = None,
) -> pd.DataFrame:
    """Process sites for a given protein id.
    
    Parameters
    ----------
    phosphosite_df : pd.DataFrame
        Phosphosite df.
    structure_df : pd.DataFrame
        Structure df.
    protein_id : List[str]
        List of protein ids to process.
    ref_atom : str, optional
        Reference atom to use for calculating distances, by default "ca"
    radius : float, optional
        Radius to filter by, by default 6.0
    adjacent_range : int, optional
        Range of adjacent residues to exclude from "next-nearest spatial
        neighbour" calculation, by default 1
        For example, if adjacent_range = 1, then residues -1 and +1 will be
        excluded from the candidates (relative to the site of interest's position).
    next_nearest : int, optional
        Number of next-nearest spatial neighbours to calculate, by default 2. 
        For example, if next_nearest = 2, then the 2 closest residues to the site
        of interest will be calculated.  If there is no residue within the radius, 
        then the value will be NaN.

    Returns
    -------
    pd.DataFrame
        Processed sites df.

    """
    if ref_atom not in ["c", "ca", "cb", "n"]:
        raise ValueError(f"Invalid reference atom: {ref_atom}")

    suffix = f"_coord_{ref_atom}"
    def get_coords(row): 
        return np.array([row[x + suffix] for x in ["x", "y", "z"]])

    if isinstance(protein_ids, str):
        protein_ids = [protein_ids]

    dict_list = []
    pbar = tqdm(enumerate(protein_ids))
    for counter, protein_id in pbar:
        pbar.set_description(protein_id)

        df = structure_df[structure_df["protein_id"] == protein_id]         # filter structure_df for this protein_id
        site_df = phosphosite_df[phosphosite_df["ACC_ID"] == protein_id]    # filter phosphosite_df for this protein_id

        # Get MOD_RSD column as list 
        mod_rsd = list(site_df["MOD_RSD"])
        mod_rsd = [x.split("-")[0] for x in mod_rsd]
        mod_rsd = [(x[0], int(x[1:])) for x in mod_rsd] # all known phosphosites for this protein
        #mod_rsd = [f"{x[0]}:{x[1:]}" for x in mod_rsd]


        if to_process == "p": 
            # Filter to just phosphorylated residues.
            to_consider = [x for x in mod_rsd if x[0] in residue_type]

        elif to_process == "all":
            # Consider all allowed residues. 
            # i.e. every residue in df that is an allowed residue. 
            to_consider = []
            for res in residue_type:
                for pos in df[df["AA"] == res]["position"].unique():
                    to_consider.append((res, pos))

        else: 
            raise ValueError(f"Invalid value for to_process: {to_process}")

        for res, pos in to_consider: 
            # Get the first row that matches the residue and position, unless there are none
            try:
                row = df[(df["AA"] == res) & (df["position"] == pos)].iloc[0]   
            except IndexError:
                if verbose: tqdm.write(f"[{protein_id}] Could not find centre residue {res} at position {pos}")
                continue
        
            site_qual = row["quality"] # store pLDDT for centre residue.
            
            site_coords = get_coords(row)
            is_phosphosite = (res, pos) in mod_rsd

            # Get the previous and next residues
            prev_dict = {}
            for i in list(range(-adjacent_range, 0)): 
                try:
                    next_row = df[(df["position"] == pos + i)].iloc[0]
                except IndexError:
                    if verbose: tqdm.write(f"[{protein_id}] Could not find residue at position {pos + i}")
                    prev_dict[f"{i}"] = np.nan
                    continue
                #coords = get_coords(next_row)
                prev_dict[f"{i}"] = next_row["AA"]+str(next_row["position"]) # (next_row["AA"], get_euc_dist(site_coords, coords))
            
            next_dict = {}
            for i in list(range(0+1, adjacent_range+1)):
                try:
                    next_row = df[(df["position"] == pos + i)].iloc[0]
                except IndexError:
                    if verbose: tqdm.write(f"[{protein_id}] Could not find residue at position {pos + i}")
                    prev_dict[f"+{i}"] = np.nan
                    continue
                
                #coords = get_coords(next_row)
                next_dict[f"+{i}"] = next_row["AA"]+str(next_row["position"]) # (next_row["AA"], get_euc_dist(site_coords, coords))
            
            # Filter df to rows within radius of site_coords. 
            candidate_df = df
            candidate_df["euc_dist"] = candidate_df.apply(lambda row: get_euc_dist(site_coords, get_coords(row)), axis=1)
            candidate_df = candidate_df[candidate_df["euc_dist"] <= radius]
            # Exclude rows with position in [pos-adjacent_range, pos+adjacent_range]
            candidate_df = candidate_df[~candidate_df["position"].isin(list(range(pos-adjacent_range, pos+adjacent_range+1)))]
            candidate_df["seq_dist"] = candidate_df["position"] - pos
            
            # sort in ascending order by euc_dist
            candidate_df = candidate_df.sort_values("euc_dist")
            nearest_dict = {}
            for i in range(1, next_nearest+1):
                # If results in IndexError, then there are less than i residues within radius
                # fill with NaN
                try:
                    next_row = candidate_df.iloc[i-1]
                except IndexError:
                    nearest_dict[f"{i}_res"] = np.nan
                    nearest_dict[f"{i}_euc_dist"] = np.nan
                    nearest_dict[f"{i}_seq_dist"] = np.nan
                    continue
                # Pick rank i of the nearest residues (euc_dist)
                nearest_dict[f"{i}_res"] = next_row["AA"] + str(next_row["position"])
                nearest_dict[f"{i}_euc_dist"] = next_row["euc_dist"]
                nearest_dict[f"{i}_seq_dist"] = next_row["seq_dist"]
            dict_list.append({
                "phosphosite": is_phosphosite,
                "site_qual": site_qual,
                "protein_id": protein_id,
                **prev_dict,
                "site": f"{res}{pos}",
                **next_dict,
                **nearest_dict,
            })
            
        # Save dataframe every 100 rows
        if counter % 100 == 0:
            print(f"Saving dataframe to {filepath}")
            pd.DataFrame(dict_list).to_csv(filepath, sep="\t", index=False)
    
    final_df = pd.DataFrame(dict_list)
    final_df.to_csv(filepath, sep="\t", index=False)
    return final_df
    # save df 


In [183]:
to_process = "all"
residue_type = "STY"
radius = 6.0
residue_adjacent = 2
next_nearest = 3
ref_atom = "ca"

out_filename = f"{to_process}-{residue_type}-{int(radius)}A{residue_adjacent}R{next_nearest}N-{ref_atom}.csv"

outfile = DATA_DIR / "motif" / out_filename
print(f"Running {outfile} ...")


Running /home/cim/STRUCTURAL_MOTIFS/phosphosite/data/motif/all-STY-6A2R3N-ca.csv ...


In [191]:
# load df 
prev_processed_df = pd.read_csv(outfile, sep="\t")
processed_ids = list(prev_processed_df["protein_id"].unique())
len(processed_ids)

100

In [194]:
unprocessed_ids = [x for x in all_protein_ids if x not in processed_ids]
len(unprocessed_ids), len(all_protein_ids)

(17236, 17336)

In [186]:
processed_df = process_sites(
    phosphosite_df, 
    structure_df, 
    protein_ids=unprocessed_ids, 
    adjacent_range=residue_adjacent,
    next_nearest=next_nearest,
    radius=radius,
    to_process=to_process, # p
    residue_type=residue_type,
    ref_atom=ref_atom,
    verbose=False,
    filepath=outfile,
)

A0A087WUL8: : 1it [00:00,  4.79it/s]

Saving dataframe to /home/cim/STRUCTURAL_MOTIFS/phosphosite/data/motif/all-STY-6A2R3N-ca.csv


A4D1P6: : 101it [04:36,  3.08s/it]  

Saving dataframe to /home/cim/STRUCTURAL_MOTIFS/phosphosite/data/motif/all-STY-6A2R3N-ca.csv


A6NFI3: : 201it [04:39, 34.04it/s]

...

O75891: : 1800it [05:15, 50.66it/s]

Saving dataframe to /home/cim/STRUCTURAL_MOTIFS/phosphosite/data/motif/all-STY-6A2R3N-ca.csv


O94823: : 1901it [05:18, 30.77it/s]

Saving dataframe to /home/cim/STRUCTURAL_MOTIFS/phosphosite/data/motif/all-STY-6A2R3N-ca.csv


O95163: : 2000it [05:20,  6.24it/s]


In [187]:
len(processed_df)

10405

In [177]:
# What proportion have phosphosite True
processed_df["phosphosite"].value_counts(normalize=True)

False    0.905815
True     0.094185
Name: phosphosite, dtype: float64

In [178]:
# What proportion have residues (i.e. first letter of 'site')
processed_df["site"].apply(lambda x: x[0]).value_counts(normalize=True)

S    0.508313
T    0.329938
Y    0.161749
Name: site, dtype: float64

In [141]:
df = processed_df

In [142]:
psite = df[df["phosphosite"] == True]
psite["site"].apply(lambda x: x[0]).value_counts(normalize=True)

S    0.598980
T    0.242857
Y    0.158163
Name: site, dtype: float64

In [143]:
notpsite = df[df["phosphosite"] == False]
notpsite["site"].apply(lambda x: x[0]).value_counts(normalize=True)

S    0.498886
T    0.338992
Y    0.162122
Name: site, dtype: float64

In [161]:
# psite rows with 1_res not NaN
psite[psite["1_res"].notna()]["site"].apply(lambda x: x[0]).value_counts(normalize=True)

S    0.479452
Y    0.264840
T    0.255708
Name: site, dtype: float64

In [162]:
for n in [1, 2, 3]:
    length = len(psite[psite[f"{str(n)}_res"].notna()])
    print(f"Number of phosphosites with >= {n} nearest residues: {length}")

Number of phosphosites with >= 1 nearest residues: 438
Number of phosphosites with >= 2 nearest residues: 307
Number of phosphosites with >= 3 nearest residues: 164


In [174]:
n = 3
cols = [f"{int(i)}_{res}" for i in range(1, n+1) for res in ["euc_dist", "seq_dist"]]
print(psite[psite[f"{int(n)}_res"].notna()][cols])

       1_euc_dist  1_seq_dist  2_euc_dist  2_seq_dist  3_euc_dist  3_seq_dist
2        4.818600       122.0    5.002838        -3.0    5.410340       121.0
426      4.805697       122.0    4.999018        -3.0    5.429360       121.0
429      4.190240        21.0    4.771475        22.0    5.035984        20.0
430      4.079906        29.0    5.360338        28.0    5.411511        30.0
431      5.217619       -34.0    5.288498       -45.0    5.296298       -35.0
...           ...         ...         ...         ...         ...         ...
10142    5.411226      -142.0    5.498299        -3.0    5.719013      -143.0
10143    4.967875      -144.0    5.666538      -146.0    5.919139      -143.0
10145    4.317854      -153.0    5.605041      -165.0    5.647070      -123.0
10170    4.263599         8.0    4.519881         9.0    5.526067        -4.0
10278    5.020814         3.0    5.176321        -3.0    5.895667        -4.0

[164 rows x 6 columns]


In [155]:
N = len(df.protein_id.unique())
fn = out_filename + f"-{N}P.csv"
outfile = DATA_DIR / "motif" / fn
# save df 
df.to_csv(outfile, index=False)